In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import time

import torchvision
from torchvision.transforms.functional import normalize
from torchvision.transforms.transforms import ToTensor
from torchvision.transforms.transforms import Resize
from torchvision.transforms.transforms import RandomHorizontalFlip
from torchvision.transforms.transforms import Normalize


PATH_TO_DATASET="/content/drive/MyDrive/chest_xray/train"

In [ ]:
def load_data(path, test_split, val_split, batch_size, input_size):
    
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )
    ######## Write your code here ########
    transform_dict={"src":transforms.Compose([
      transforms.Resize((224,224)),
            transforms.ToTensor(),
            normalize])}
      
    data=torchvision.datasets.ImageFolder(root=PATH_TO_DATASET,transform=transform_dict["src"])
#dataset = TensorDataset(x_tensor, y_tensor)
#val_size = int(len(dataset)*0.2)
#train_size = len(dataset)- int(len(dataset)*0.2)
#train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_split=1-test_split-val_split
    train_size=int(len(data)*train_split)
    val_size=int(len(data)*val_split)
    test_size=int(len(data)*test_split)
    print("The train size, val size and test size is resp ",train_size," ",val_size," ",test_size," ",len(data))
    train,val,test=torch.utils.data.random_split(data,[train_size,val_size,test_size])
    data_loader_train = torch.utils.data.DataLoader(train, batch_size=batch_size,
                                          shuffle=True, num_workers=0, drop_last=False)
    data_loader_test = torch.utils.data.DataLoader(test, batch_size=batch_size,
                                          shuffle=True, num_workers=0, drop_last=False)
    data_loader_val = torch.utils.data.DataLoader(val, batch_size=batch_size,
                                          shuffle=True, num_workers=0, drop_last=False)
    return data_loader_train, data_loader_test, data_loader_val


In [ ]:
data_loader_train, data_loader_test, data_loader_val=load_data(PATH_TO_DATASET,0.25,0.25,32,40)

The train size, val size and test size is resp  2928   1464   1464   5856


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = 3
num_epochs = 20
batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  

# Train the model
total_step = len(data_loader_train)

In [ ]:
import gc
total_step = len(data_loader_train)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader_train):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()

    print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in data_loader_val:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 

Epoch [1/20], Loss: 0.5535
Accuracy of the network on the 5000 validation images: 65.1639344262295 %
Epoch [2/20], Loss: 1.2063
Accuracy of the network on the 5000 validation images: 73.90710382513662 %
Epoch [3/20], Loss: 0.9346
Accuracy of the network on the 5000 validation images: 74.4535519125683 %
Epoch [4/20], Loss: 0.8267
Accuracy of the network on the 5000 validation images: 77.8688524590164 %
Epoch [5/20], Loss: 0.3682
Accuracy of the network on the 5000 validation images: 79.57650273224044 %
Epoch [6/20], Loss: 1.2828
Accuracy of the network on the 5000 validation images: 70.69672131147541 %
Epoch [7/20], Loss: 0.4660
Accuracy of the network on the 5000 validation images: 76.77595628415301 %
Epoch [8/20], Loss: 0.3360
Accuracy of the network on the 5000 validation images: 77.3224043715847 %
Epoch [9/20], Loss: 0.3886
Accuracy of the network on the 5000 validation images: 76.09289617486338 %
Epoch [10/20], Loss: 0.3180
Accuracy of the network on the 5000 validation images: 78.

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in data_loader_test:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))   

Accuracy of the network on the 10000 test images: 76.36612021857924 %
